# Imports

In [1]:
import numpy as np

from data_worker.data_worker import unpickle, unpack_data, combine_batches, split_into_batches
from tf_lib.data_worker import suit4tf
from tf_lib.Interface import Interface
from tf_lib.Nets import LargeNet, MediumNet, SmallNet

# Data

In [2]:
batches_names = [
    'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4',
    'data_batch_5'
]

data_batches = [
    unpickle(f'datasets/cifar-10-batches-py/{batch_name}') for batch_name
    in batches_names]

unpacked_batches = [
    (unpack_data(data_batch)) for data_batch
    in data_batches]

print(unpacked_batches[0][0].shape)

X, Y = combine_batches(unpacked_batches)

X_tf, Y_tf = suit4tf(X, Y)

batches = split_into_batches(X_tf, Y_tf, 3)

# torch_batches = [(suit4torch(X, Y)) for X, Y in batches]

X_batch0 = batches[0][0]
Y_batch0 = batches[0][1]

print(X.shape, Y.shape)

(10000, 32, 32, 3)
(32, 32, 3)
(50000, 32, 32, 3) (50000,)


# Train network

In [33]:
net = SmallNet
# net = MediumNet
# net = LargeNet

In [34]:
net_interface = Interface(net)

In [35]:
net_interface.train_net([(X_tf, Y_tf)], 1, batch_size=10)

5000/5000 [==============================] - 40s 8ms/step - loss: 1.5279 - accuracy: 0.4487


# Save weights

In [36]:
PATH = 'saved_nets/saved_tf/small_v1.pth'
# PATH = 'saved_nets/saved_tf/medium_v1.pth'
# PATH = 'saved_nets/saved_tf/large_v1.pth'

In [37]:
net_interface.save_weights(PATH)

# Predict

In [9]:
preds = net_interface.predict_net(X_batch0)

In [10]:
print(preds)
print(np.argmax(preds, axis=1), Y_batch0)

[[-3.5743718  -3.0529528  -0.4828685   1.554341    1.0313882   0.5706348
   3.4274993  -0.09871575 -3.714269   -2.160081  ]
 [ 0.5407109   4.955052   -0.0794224  -0.85406804 -2.3996065  -1.1269073
  -1.0047617  -1.7425171   1.1630034   5.3968883 ]
 [ 3.2598748   3.0464036   0.15074204  0.15174165 -0.5821962  -2.0008912
  -1.0348898   2.1388137   2.1042237   7.2403584 ]]
[6 9 9] tf.Tensor([6. 9. 9.], shape=(3,), dtype=float32)


# Evaluate Accuracy

In [12]:
acc, N = net_interface.eval_acc_net(X_tf, Y_tf)
print(acc, N)

0.53776 50000


# Load weights

In [29]:
net_interface.load_weights(PATH)

In [31]:
preds = net_interface.predict_net(X_tf)

In [32]:
print(preds)
print(np.argmax(preds, axis=1), Y_tf)

[[-3.5743718e+00 -3.0529528e+00 -4.8286861e-01 ... -9.8715618e-02
  -3.7142687e+00 -2.1600809e+00]
 [ 5.4071140e-01  4.9550519e+00 -7.9422593e-02 ... -1.7425169e+00
   1.1630034e+00  5.3968883e+00]
 [ 3.2598748e+00  3.0464041e+00  1.5074250e-01 ...  2.1388137e+00
   2.1042230e+00  7.2403579e+00]
 ...
 [ 2.9858768e+00  3.2573800e+00 -6.1973691e-01 ...  3.6352268e-01
   2.8308268e+00  5.1723042e+00]
 [ 2.8082774e+00  6.7876973e+00 -8.4085643e-01 ... -3.0735955e+00
   6.0954432e+00  6.0286546e+00]
 [ 3.1594229e+00  1.3739679e+00  3.2654777e-03 ... -3.1382853e-01
   2.9303303e+00  1.9908476e+00]]
[6 9 9 ... 9 1 0] tf.Tensor([6. 9. 9. ... 9. 1. 1.], shape=(50000,), dtype=float32)


# Run forward function

In [14]:
from threading import Thread
import time
import psutil
import subprocess
import pandas as pd


class GpuReader(Thread):
    
    def __init__(self):
        self.process = subprocess.Popen(['tegrastats'], stdout=subprocess.PIPE)
        self.stopped = True
        self.values = {}
        super().__init__()
    
    def __enter__(self):
        self.start()
        return self
    
    def __exit__(self, exception_type, exception_value, traceback):
        self.stop()
    
    def start(self):
        self.stopped = False
        super().start()
    
    def stop(self):
        self.stopped = True
    
    def run(self):
        while not self.stopped:
            resp = self.process.stdout.readline().strip().decode('utf-8')
            resp_array = resp.split(' ')
            idx = resp_array.index('GR3D_FREQ')
            self.values['GR3D_FRWQ'] = resp_array[idx + 1]


class CpuGpuTracker(Thread):
    
    def __init__(self, Ts):
        self.Ts = Ts
        self.stopped = True
        self.start_time = None
        self.values = []
        super().__init__()
    
    def __enter__(self):
        self.start()
        return self
    
    def __exit__(self, exception_type, exception_value, traceback):
#         print(exception_type, exception_value, traceback)
        self.stop()
    
    def start(self):
        self.stopped = False
        self.start_time = time.time()
        super().start()
    
    def stop(self):
        self.stopped = True

    def run(self):
        while not self.stopped:
            mem = psutil.virtual_memory()
            cpu_percent = psutil.cpu_percent()
            gpu_percent = 20
            print(mem)
            print(f'{time.time():.8f}')
            self.values.append((time.time() - self.start_time, mem.percent, cpu_percent, gpu_percent))
            time.sleep(self.Ts)
    
    def get_values(self):
        return pd.DataFrame(self.values, columns=['time', 'memory', 'cpu_percent', 'gpu_percent'])        
    

In [17]:
import sys
import os
import torch
import time


def execute_net(net_interface, X_data, Y_data, batch_size, priority, loops=1, device='cuda', metrics=['ex_time', 'cpu_max', 'gpu_max', 'mem_max'], echo=True):
    if sys.platform == 'linux':
        os.nice(priority)

    if device not in ['cuda', 'cpu']:
        raise Exception("'device' parameter must be one of 'cuda' or 'cpu'")
    if device == 'cpu':
        os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

    batches = split_into_batches(X_data, Y_data, batch_size)
    
    batch_exec_times = []

    procesor_tracked_values = None
    with CpuGpuTracker(0.1) as tracker:
        
        initial_time = time.time()
    
        for loop in range(loops):
            if echo:
                print('loop:', loop)
            batch_count = 0
            for X_batch, Y_batch in batches:
                start_time = time.time()
                Y_pred = net_interface.predict_net(X_batch)
                batch_time = time.time() - start_time
                if echo:
                    print(f'batch_time: {batch_time:.8f}')
                batch_exec_times.append((loop, batch_count, batch_time, time.time() - initial_time))
                batch_count += 1
        
        procesor_tracked_values = tracker.get_values()
    
    batch_exec_times = pd.DataFrame(batch_exec_times, columns=['loop', 'batch_count', 'batch_time', 'time'])
    return procesor_tracked_values, batch_exec_times

tracked_values, batch_exec_times = execute_net(net_interface, X_tf, Y_tf, 10, 0)


svmem(total=8390193152, available=2046750720, percent=75.6, used=6343442432, free=2046750720)loop: 0

1634661962.25859332
batch_time: 0.04689312
svmem(total=8390193152, available=2045345792, percent=75.6, used=6344847360, free=2045345792)
1634661962.36825991
batch_time: 0.06606579
batch_time: 0.04303384
batch_time: 0.03125262
svmem(total=8390193152, available=2048380928, percent=75.6, used=6341812224, free=2048380928)
1634661962.48256588
batch_time: 0.07187009
batch_time: 0.05992317
svmem(total=8390193152, available=2037604352, percent=75.7, used=6352588800, free=2037604352)
1634661962.59594941
batch_time: 0.03700376
batch_time: 0.04682779
batch_time: 0.03802228
svmem(total=8390193152, available=2033967104, percent=75.8, used=6356226048, free=2033967104)
1634661962.71510792
batch_time: 0.06441593
batch_time: 0.03264999
svmem(total=8390193152, available=2037268480, percent=75.7, used=6352924672, free=2037268480)
1634661962.82781935
batch_time: 0.04685855
batch_time: 0.04686260
svmem(tot

svmem(total=8390193152, available=2028064768, percent=75.8, used=6362128384, free=2028064768)
1634661968.38867116
batch_time: 0.07080698
batch_time: 0.06685519
svmem(total=8390193152, available=2034438144, percent=75.8, used=6355755008, free=2034438144)
1634661968.49740291
batch_time: 0.06282544
batch_time: 0.06482601
svmem(total=8390193152, available=2034991104, percent=75.7, used=6355202048, free=2034991104)
1634661968.60708761
batch_time: 0.07180834
svmem(total=8390193152, available=2024574976, percent=75.9, used=6365618176, free=2024574976)
1634661968.71779156
batch_time: 0.07280421
batch_time: 0.06881785
svmem(total=8390193152, available=2031849472, percent=75.8, used=6358343680, free=2031849472)
1634661968.82649994
batch_time: 0.07080984
svmem(total=8390193152, available=2037219328, percent=75.7, used=6352973824, free=2037219328)
1634661968.93421435
batch_time: 0.06981254
batch_time: 0.06482959
svmem(total=8390193152, available=2027712512, percent=75.8, used=6362480640, free=2027

svmem(total=8390193152, available=2020511744, percent=75.9, used=6369681408, free=2020511744)
1634661974.11692834
batch_time: 0.06369209
batch_time: 0.04358721
batch_time: 0.03124809
svmem(total=8390193152, available=2007883776, percent=76.1, used=6382309376, free=2007883776)
1634661974.22334003
batch_time: 0.04686427
batch_time: 0.04686308
svmem(total=8390193152, available=2010914816, percent=76.0, used=6379278336, free=2010914816)
1634661974.33525181
batch_time: 0.05862451
batch_time: 0.04686594
svmem(total=8390193152, available=2014154752, percent=76.0, used=6376038400, free=2014154752)
1634661974.43818307
batch_time: 0.04686594
batch_time: 0.04686141
svmem(total=8390193152, available=2017382400, percent=76.0, used=6372810752, free=2017382400)
1634661974.54753041
batch_time: 0.05253434
batch_time: 0.04010224
batch_time: 0.04686809
svmem(total=8390193152, available=2006753280, percent=76.1, used=6383439872, free=2006753280)
1634661974.65579081
batch_time: 0.04685998
batch_time: 0.046

batch_time: 0.06682873
svmem(total=8390193152, available=2007130112, percent=76.1, used=6383063040, free=2007130112)
1634661980.16685963
batch_time: 0.06582308
svmem(total=8390193152, available=2000195584, percent=76.2, used=6389997568, free=2000195584)
1634661980.27656698
batch_time: 0.06682563
batch_time: 0.06482339
svmem(total=8390193152, available=2007379968, percent=76.1, used=6382813184, free=2007379968)
1634661980.38726854
batch_time: 0.06881452
svmem(total=8390193152, available=2002882560, percent=76.1, used=6387310592, free=2002882560)
1634661980.49498320
batch_time: 0.07380390
batch_time: 0.06383562
svmem(total=8390193152, available=2001440768, percent=76.1, used=6388752384, free=2001440768)
1634661980.60568690
batch_time: 0.07080889
batch_time: 0.06881022
svmem(total=8390193152, available=2009075712, percent=76.1, used=6381117440, free=2009075712)
1634661980.71439838
batch_time: 0.06183553
svmem(total=8390193152, available=1999572992, percent=76.2, used=6390620160, free=1999

svmem(total=8390193152, available=1991221248, percent=76.3, used=6398971904, free=1991221248)
1634661986.34299922
batch_time: 0.08178043
svmem(total=8390193152, available=1988018176, percent=76.3, used=6402174976, free=1988018176)
1634661986.44971347
batch_time: 0.07480216
batch_time: 0.07978487
svmem(total=8390193152, available=1986654208, percent=76.3, used=6403538944, free=1986654208)
1634661986.56440735
batch_time: 0.08676624
svmem(total=8390193152, available=1992921088, percent=76.2, used=6397272064, free=1992921088)
1634661986.67012358
batch_time: 0.08377576
svmem(total=8390193152, available=1985806336, percent=76.3, used=6404386816, free=1985806336)
1634661986.77883554
batch_time: 0.08078337
batch_time: 0.07679510
svmem(total=8390193152, available=1982251008, percent=76.4, used=6407942144, free=1982251008)
1634661986.89054346
batch_time: 0.06981421
svmem(total=8390193152, available=1983860736, percent=76.4, used=6406332416, free=1983860736)
1634661986.99924612
batch_time: 0.0847

...

batch_time: 0.05970454
svmem(total=8390193152, available=1826070528, percent=78.2, used=6564122624, free=1826070528)
1634662257.70558429
batch_time: 0.04594970
batch_time: 0.06248784
svmem(total=8390193152, available=1829109760, percent=78.2, used=6561083392, free=1829109760)
1634662257.81069946
batch_time: 0.04689956
batch_time: 0.06592536
svmem(total=8390193152, available=1817067520, percent=78.3, used=6573125632, free=1817067520)
1634662257.92285585
batch_time: 0.04096150
batch_time: 0.06248927
svmem(total=8390193152, available=1820315648, percent=78.3, used=6569877504, free=1820315648)
1634662258.02697873
batch_time: 0.04689360
batch_time: 0.06882668
svmem(total=8390193152, available=1823580160, percent=78.3, used=6566612992, free=1823580160)
1634662258.13804722
batch_time: 0.05419207
batch_time: 0.04686499
svmem(total=8390193152, available=1826828288, percent=78.2, used=6563364864, free=1826828288)
1634662258.24375010
batch_time: 0.06248403
batch_time: 0.05835056
svmem(total=83901

svmem(total=8390193152, available=1828761600, percent=78.2, used=6561431552, free=1828761600)
1634662263.36365652
batch_time: 0.05892396
batch_time: 0.04689598
svmem(total=8390193152, available=1823449088, percent=78.3, used=6566744064, free=1823449088)
1634662263.47422481
batch_time: 0.04683828
batch_time: 0.06251240
svmem(total=8390193152, available=1821753344, percent=78.3, used=6568439808, free=1821753344)
1634662263.58681083
batch_time: 0.06008697
batch_time: 0.04686666
svmem(total=8390193152, available=1825001472, percent=78.2, used=6565191680, free=1825001472)
1634662263.69052792
batch_time: 0.06248569
batch_time: 0.04689717
svmem(total=8390193152, available=1828253696, percent=78.2, used=6561939456, free=1828253696)
1634662263.79987717
batch_time: 0.07287264
batch_time: 0.05329323
svmem(total=8390193152, available=1819910144, percent=78.3, used=6570283008, free=1819910144)
1634662263.91045570
batch_time: 0.04689598
batch_time: 0.06245351
svmem(total=8390193152, available=182316

svmem(total=8390193152, available=1830879232, percent=78.2, used=6559313920, free=1830879232)
1634662269.05000448
batch_time: 0.06805134
batch_time: 0.05460095
svmem(total=8390193152, available=1834131456, percent=78.1, used=6556061696, free=1834131456)
1634662269.29761171
svmem(total=8390193152, available=1834332160, percent=78.1, used=6555860992, free=1834332160)
1634662269.42233348
batch_time: 0.34344959
batch_time: 0.05741429
svmem(total=8390193152, available=1837522944, percent=78.1, used=6552670208, free=1837522944)
1634662269.53392649
batch_time: 0.04893231
batch_time: 0.04689765
svmem(total=8390193152, available=1840771072, percent=78.1, used=6549422080, free=1840771072)
1634662269.63905454
batch_time: 0.06245852
batch_time: 0.06285119
svmem(total=8390193152, available=1828442112, percent=78.2, used=6561751040, free=1828442112)
1634662269.74908495
batch_time: 0.04396248
batch_time: 0.06248689
svmem(total=8390193152, available=1829466112, percent=78.2, used=6560727040, free=1829

batch_time: 0.06173110
svmem(total=8390193152, available=1828474880, percent=78.2, used=6561718272, free=1828474880)
1634662275.23494554
batch_time: 0.06139064
batch_time: 0.05542040
svmem(total=8390193152, available=1831727104, percent=78.2, used=6558466048, free=1831727104)
1634662275.34576821
batch_time: 0.04591799
svmem(total=8390193152, available=1833353216, percent=78.1, used=6556839936, free=1833353216)
1634662275.45403409
batch_time: 0.07531381
batch_time: 0.06200242
svmem(total=8390193152, available=1822912512, percent=78.3, used=6567280640, free=1822912512)
1634662275.56789732
batch_time: 0.06607842
batch_time: 0.05589032
svmem(total=8390193152, available=1822146560, percent=78.3, used=6568046592, free=1822146560)
1634662275.67492533
batch_time: 0.06369448
batch_time: 0.04340053
svmem(total=8390193152, available=1825435648, percent=78.2, used=6564757504, free=1825435648)
1634662275.79152656
batch_time: 0.06659865
batch_time: 0.06516528
svmem(total=8390193152, available=182874

batch_time: 0.06245589
svmem(total=8390193152, available=1817825280, percent=78.3, used=6572367872, free=1817825280)
1634662281.20399618
batch_time: 0.07400203
batch_time: 0.04761028
svmem(total=8390193152, available=1820528640, percent=78.3, used=6569664512, free=1820528640)
1634662281.31999350
batch_time: 0.06248331
svmem(total=8390193152, available=1822154752, percent=78.3, used=6568038400, free=1822154752)
1634662281.42929459
batch_time: 0.07156706
batch_time: 0.04686689
svmem(total=8390193152, available=1819344896, percent=78.3, used=6570848256, free=1819344896)
1634662281.53217387
batch_time: 0.09002280
svmem(total=8390193152, available=1811869696, percent=78.4, used=6578323456, free=1811869696)
1634662281.64911652
batch_time: 0.08577204
svmem(total=8390193152, available=1813254144, percent=78.4, used=6576939008, free=1813254144)
1634662281.75782871
batch_time: 0.09973311
batch_time: 0.07978582
svmem(total=8390193152, available=1860304896, percent=77.8, used=6529888256, free=1860

In [18]:
tracked_values

,time,memory,cpu_percent,gpu_percent
0,0.015621,75.6,15.9,20
1,0.125288,75.6,21.4,20
2,0.239594,75.6,39.3,20
3,0.352977,75.7,28.6,20
4,0.472136,75.8,31.2,20
...,...,...,...,...
2864,319.289201,78.3,48.0,20
2865,319.406144,78.4,89.3,20
2866,319.514856,78.4,97.0,20
2867,319.622565,77.8,80.8,20


In [19]:
batch_exec_times

,loop,batch_count,batch_time,time
0,0,0,0.046893,0.046893
1,0,1,0.066066,0.112959
2,0,2,0.043034,0.155993
3,0,3,0.031253,0.187245
4,0,4,0.071870,0.259115
...,...,...,...,...
4995,0,4995,0.099733,319.502225
4996,0,4996,0.079786,319.583008
4997,0,4997,0.069813,319.653819
4998,0,4998,0.071005,319.724824
